In [1]:
import pandas as pd
import numpy as np
import itertools
import lightgbm as lgbm
import matplotlib.pyplot as plt

from script import *
from feature_engineering import *
import db_operations as dbop
from constants import *
import data_process as dp

import time

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
pd.set_option('display.max_rows',200)
idx = pd.IndexSlice
agg_operations =['mean','median','max','min','std','size']

In [3]:
%time df = pd.read_hdf(os.path.join('datasets','df_20140101.h5'),key='data')
df = df.sort_index()
df.shape
df.columns

Wall time: 3.94 s


(4593560, 64)

Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', 'turnover_rate', 'turnover_rate_f',
       'volume_ratio', 'pe', 'pe_ttm', 'pb', 'ps', 'ps_ttm', 'total_share',
       'float_share', 'free_share', 'total_mv', 'circ_mv', 'pb*pe_ttm',
       'pb*pe', '5ma', '10ma', '20ma', '40ma', '60ma', '120ma', '250ma',
       'close/5ma', 'close/10ma', 'close/20ma', 'close/40ma', 'close/60ma',
       'close/120ma', 'close/250ma', '60ma_pct', '20ma_pct', 'p5max_abs_pct',
       'p10max_abs_pct', '30ma', '30ma_pct', 'close/30ma', 'prev_close',
       'day_pct', 'close0_rank', 'close0_pct', 'prev_pct', 'prev_day_pct',
       'sell_at', 'sell_at0', 'sell_date', 'sell_idx', 'buy_at', 'buy_at0',
       'max', 'max_idx', 'idx', 'is_selled', 'r'],
      dtype='object')

In [4]:
min_periods=1
periods = [3]
for k in periods:
    %time df['{:d}ma'.format(k)] = groupby_rolling(df[['close']],level="code",window=k,ops='mean',min_periods=min_periods)

Wall time: 7.11 s


In [5]:
%time df['p3mean_amt'] = groupby_rolling(df[['amt']],level="code",window=3,ops='mean',min_periods=min_periods)

Wall time: 7.57 s


In [6]:
tmp = pd.DataFrame(index=df.index)
tmp[0]=1
%time tmp['idx']=tmp.reset_index('code').groupby('code').expanding()[0].sum()
tmp.head(20)
%time df['idx'] = tmp['idx']

Wall time: 4.35 s


0   idx
code      date               
000001.SZ 2014-01-02  1   1.0
          2014-01-03  1   2.0
          2014-01-06  1   3.0
          2014-01-07  1   4.0
          2014-01-08  1   5.0
          2014-01-09  1   6.0
          2014-01-10  1   7.0
          2014-01-13  1   8.0
          2014-01-14  1   9.0
          2014-01-15  1  10.0
          2014-01-16  1  11.0
          2014-01-17  1  12.0
          2014-01-20  1  13.0
          2014-01-21  1  14.0
          2014-01-22  1  15.0
          2014-01-23  1  16.0
          2014-01-24  1  17.0
          2014-01-27  1  18.0
          2014-01-28  1  19.0
          2014-01-29  1  20.0

Wall time: 23 ms


In [7]:
target_col = 'p3mean_amt'
target_ss = df[target_col].fillna(0)

# target_ix = np.nonzero(df.columns==target_col)[0][0]
# idx_ix = np.nonzero(df.columns=='idx')[0][0]
# target_ix,idx_ix
# df.columns[[target_ix,idx_ix]]


# def idxmax_inc_1(x:np.array):
#     return np.argmax(np.nan_to_num(x.astype(float)))+1
# # df['']df.groupby('code')[target_col].rolling(20).agg(idxmax_inc_1)


for k in [20,40]:
#     %time df[['p{}max_p3mean_amt'.format(k),'p{}idxmax_p3mean_amt'.format(k)]] = groupby_rolling(df[[target_col]],level="code",window=k,ops=['max',idxmax_inc_1],min_periods=min_periods)
    %time tmp = groupby_rolling(df[[target_col,'close']],level="code",window=k,ops=['max'],min_periods=min_periods)
    df[tmp.columns] = tmp

#     df[['p{}max_p3mean_amt'.format(k),'p{}max_close'.format(k)]].head(50)
#     %time df['p{}idxmax_p3mean_amt'.format(k)]= (k-1)-target_ss.rolling(k).apply(lambda x:np.argmax(x))
#     df_check = pd.DataFrame(index=df.index)
#     df_check['mark']=1
#     mask = df_check.groupby('code')['mark'].shift(k - 1).isnull()
#     df.loc[mask,'p{}idxmax_p3mean_amt'.format(k)]=np.nan

Wall time: 7.56 s
Wall time: 7.65 s


In [144]:
# df[['p{}idxmax_p3mean_amt'.format(k),'p{}max_p3mean_amt'.format(k),target_col,'idx']].head(100)

df[['p{}max_p3mean_amt'.format(k),'p{}max_close'.format(k)]].head(50)

p20max_p3mean_amt  p20max_close
code      date                                       
000001.SZ 2014-01-02                NaN           NaN
          2014-01-03                NaN           NaN
          2014-01-06                NaN           NaN
          2014-01-07                NaN           NaN
          2014-01-08                NaN           NaN
          2014-01-09                NaN           NaN
          2014-01-10                NaN           NaN
          2014-01-13                NaN           NaN
          2014-01-14                NaN           NaN
          2014-01-15                NaN           NaN
          2014-01-16                NaN           NaN
          2014-01-17                NaN           NaN
          2014-01-20                NaN           NaN
          2014-01-21                NaN           NaN
          2014-01-22                NaN           NaN
          2014-01-23                NaN           NaN
          2014-01-24                NaN           NaN
          2014-01-27                NaN           NaN
          2014-01-28                NaN           NaN
          2014-01-29           6.440451    714.072998
          2014-01-30           6.440451    696.556946
          2014-02-07           6.440451    690.134338
          2014-02-10           5.768454    690.134338
          2014-02-11           9.032334    700.644043
          2014-02-12          10.237650    700.644043
          2014-02-13          12.939544    713.489197
          2014-02-14          12.939544    713.489197
          2014-02-17          12.939544    713.489197
          2014-02-18          12.939544    713.489197
          2014-02-19          12.939544    713.489197
          2014-02-20          12.939544    713.489197
          2014-02-21          12.939544    713.489197
          2014-02-24          12.939544    713.489197
          2014-02-25          12.939544    713.489197
          2014-02-26          12.939544    713.489197
          2014-02-27          12.939544    713.489197
          2014-02-28          12.939544    713.489197
          2014-03-03          12.939544    713.489197
          2014-03-04          12.939544    713.489197
          2014-03-05          12.939544    713.489197
          2014-03-06          12.939544    713.489197
          2014-03-07          12.939544    713.489197
          2014-03-10          12.939544    713.489197
          2014-03-11          12.939544    713.489197
          2014-03-12          12.939544    713.489197
          2014-03-13          10.047386    705.314941
          2014-03-14           9.700279    700.644043
          2014-03-17           9.323386    700.644043
          2014-03-18           9.323386    700.644043
          2014-03-19           9.323386    686.631165

In [127]:
mask_date = df.index.get_level_values('date')>='2016-01-01'
df_baseline = df.loc[mask_date,'r'].agg(agg_operations[:-1])
df_baseline_q = df.loc[mask_date,'r'].reset_index('code').resample('Q').agg(agg_operations)
df_baseline
df_baseline_q

mean     -0.003319
median   -0.021764
max       5.543209
min      -0.972953
std       0.094291
Name: r, dtype: float64

r                                                
                mean    median       max       min       std    size
date                                                                
2016-03-31  0.001993 -0.018907  3.210775 -0.972953  0.106063  150433
2016-06-30  0.001559 -0.020861  1.944000 -0.714186  0.094675  155157
2016-09-30  0.007954 -0.008368  2.405940 -0.651400  0.083539  170115
2016-12-31 -0.007982 -0.018912  2.487285 -0.900237  0.071205  165772
2017-03-31  0.000778 -0.012563  1.463308 -0.482470  0.076877  170618
2017-06-30 -0.013874 -0.030237  5.543209 -0.852388  0.088826  178502
2017-09-30  0.007274 -0.008121  1.925995 -0.804020  0.080761  200686
2017-12-31 -0.021151 -0.031991  1.990148 -0.814664  0.063842  193463
2018-03-31 -0.013712 -0.029727  1.814716 -0.967228  0.089663  190955
2018-06-30 -0.027349 -0.036075  1.174034 -0.950667  0.068656  197214
2018-09-30 -0.026512 -0.033389  1.554622 -0.869820  0.057996  218246
2018-12-31  0.002983 -0.020253  2.526171 -0.612903  0.094861  210707
2019-03-31  0.065352  0.012469  2.550678 -0.746753  0.163316  206870
2019-06-30 -0.015648 -0.027737  2.430079 -0.861940  0.070399  215321
2019-09-30 -0.000834 -0.021739  2.231214 -0.876993  0.084032  236958
2019-12-31  0.009239 -0.009906  2.763278 -0.731959  0.107321  225939
2020-03-31 -0.018712 -0.030406  2.382353 -0.763889  0.113561  218582
2020-06-30 -0.015541 -0.027642  1.050950 -0.787500  0.063938  148256

In [111]:
r_columns = ['buy_at','max','max_idx', 'idx', 'is_selled', 'sell_at', 'sell_date', 'r']


mask_shrink_amt = [
    (df['day_pct']>4) | (df['pct']>4),
#     df['amt']>df['p3mean_amt']*0.7,
#     df['amt']>df_pct_amt5['p5mean_abs_amt']*1,
    
#     (df['p40max_p3mean_amt']/df['p3mean_amt'])>1.2,
#     (df['p20max_p3mean_amt']/df['p3mean_amt'])<2,
#     (df['p40max_close']/df['close']-1)<0.1
]



# mask_low_pos = [
#     df[['close/60ma','close/30ma']].min(axis=1)<1.03,
# #     df[['close/60ma','close/30ma']].max(axis=1)<1.05,
    
#     df['close/5ma']<1.03,
# #     df['close/10ma']<1.03,
# #     df['close/20ma']<1.03,
    
# #     df['close/120ma']>1.07,
# #     df['close/250ma']>1.1,
# #     df[['close/30ma','close/60ma','close/120ma','close/250ma']].max(axis=1)>0.9,
# ]

mask_low_pos = [
    df['close/60ma']<1.05,
    df['close/30ma']<1.05,
    df['close/5ma']<1.15,
#     df['close/120ma']<1.07,
    df[['close/30ma','close/60ma','close/120ma','close/250ma']].max(axis=1)>0.9,
]


mask_basic =[   
    # 流动性、市值等基本限制 
#     mean     -2.309262e-03
#     median   -1.832455e-02
#     size   2373852
    df['amt']>=2,
    df['turnover_rate']<10,
    df['circ_mv']>50e4, 
    df['total_mv']>100e4,
    
#     (df['pb']<10) | (df['pe']>0),
    
    
#     df['close0']>5,
#     df['close0_rank']<100,
#     df['close0_rank']>10,
    df['close0_pct']<0.015,
    df['close0_pct']>0.01,

    df.index.get_level_values('code').str[:3]!='688',
]


mask_trend = [   
#     df[['60ma_pct','30ma_pct']].max(axis=1)>-0.5,
#     df['30ma_pct']>-0.2,        
]
mask_final = True
# masks = mask_big_rise+mask_basic+mask_trend
masks = mask_shrink_amt+mask_low_pos+mask_trend+mask_basic
# masks = mask_shrink_amt+mask_trend+mask_basic


for m in masks+[mask_date]:

    mask_final &= m
    m.sum(),len(m),round(m.sum()/len(m),4),round(mask_final.sum()/len(mask_final),4)

# df_baseline
# df_baseline_q


df.loc[mask_final,'r'].agg(agg_operations)
df.loc[mask_final,'r'].reset_index('code').resample('Q').agg(agg_operations)
# df.loc[mask_final,'r'].reset_index('code').groupby('date').agg(agg_operations)

tmp = df.loc[mask_final,r_columns].sort_values('date',ascending=False)
# tmp[tmp['r'].notnull()].iloc[len(tmp)//2:len(tmp)//2+50]
tmp.sort_values('date').tail(70)

(434672, 4593560, 0.0946, 0.0946)

(2716624, 4593560, 0.5914, 0.0293)

(3384975, 4593560, 0.7369, 0.0236)

(4511129, 4593560, 0.9821, 0.0233)

(4110067, 4593560, 0.8947, 0.0188)

...

(4580262, 4593560, 0.9971, 0.0)

(3453794, 4593560, 0.7519, 0.0)

mean        0.038116
median     -0.008111
max         0.518881
min        -0.133762
std         0.131119
size      109.000000
Name: r, dtype: float64

r                                             
                mean    median       max       min       std size
date                                                             
2016-03-31  0.022630 -0.006962  0.180669 -0.090278  0.104613    5
2016-06-30  0.125223  0.050122  0.518881 -0.090428  0.217829    6
2016-09-30 -0.061965 -0.080661 -0.013268 -0.091967  0.042551    3
2016-12-31       NaN       NaN       NaN       NaN       NaN    0
2017-03-31       NaN       NaN       NaN       NaN       NaN    0
2017-06-30       NaN       NaN       NaN       NaN       NaN    0
2017-09-30       NaN       NaN       NaN       NaN       NaN    0
2017-12-31 -0.032944 -0.032944 -0.022409 -0.043478  0.014898    2
2018-03-31  0.006651 -0.009041  0.160402 -0.094437  0.089821    6
2018-06-30  0.020822  0.020822  0.020822  0.020822       NaN    1
2018-09-30 -0.019088 -0.025974  0.074561 -0.077212  0.042072   11
2018-12-31 -0.018408 -0.049793  0.190527 -0.096958  0.077168   15
2019-03-31  0.136237  0.084163  0.455871 -0.046214  0.169287    9
2019-06-30  0.034525  0.010479  0.283328 -0.076462  0.107861   11
2019-09-30  0.207543  0.207543  0.422051 -0.006966  0.303361    2
2019-12-31  0.050344  0.050344  0.126238 -0.025551  0.107331    2
2020-03-31  0.064795  0.016091  0.302875 -0.133762  0.142372   25
2020-06-30  0.031432 -0.019029  0.294407 -0.078219  0.132145   11

buy_at          max  max_idx     idx  is_selled  \
code      date                                                               
600570.SH 2018-10-22   753.810608   760.546204   2071.0  1134.0       True   
600763.SH 2018-10-22   119.228546   123.406494   1976.0  1015.0       True   
600009.SH 2018-10-22   166.900436   169.796753   2135.0  1171.0       True   
          2018-10-30   144.242767   163.973938   2144.0  1177.0       True   
600570.SH 2018-10-30   665.674988   843.379333   2089.0  1140.0       True   
600009.SH 2018-11-02   163.973938   163.973938   2144.0  1180.0       True   
000858.SZ 2018-11-02   896.536499   900.622498   2083.0  1120.0       True   
600570.SH 2018-11-02   716.549988   843.379333   2089.0  1143.0       True   
601888.SH 2018-11-02   125.457634   125.500755   2081.0  1124.0       True   
          2018-11-28   120.671318   136.129837   2109.0  1142.0       True   
600276.SH 2019-01-04  1990.062012  2228.132324   2193.0  1222.0       True   
002594.SZ 2019-01-04    51.663601    54.498241   1819.0  1213.0       True   
600570.SH 2019-01-04   771.007812   902.566406   2130.0  1186.0       True   
000858.SZ 2019-01-15   898.919983  1353.487427   2161.0  1170.0       True   
600009.SH 2019-01-18   149.341492   153.173096   2197.0  1233.0       True   
601888.SH 2019-01-24   119.118996   142.382233   2155.0  1181.0       True   
600276.SH 2019-01-29  2100.621094  2727.122070   2218.0  1239.0       True   
300676.SZ 2019-02-01    60.299999    86.641052    387.0   372.0       True   
601888.SH 2019-02-11   124.616791   142.382233   2155.0  1188.0       True   
002371.SZ 2019-05-09   299.599579   346.213440   2150.0  1232.0       True   
300226.SZ 2019-05-10   279.132660   280.980713   1812.0  1193.0       True   
600570.SH 2019-05-15  1229.777832  1241.746460   2207.0  1270.0       True   
002714.SZ 2019-05-31   492.895020   517.843079   1276.0  1276.0       True   
300347.SZ 2019-06-10   510.233673   685.353455   1531.0  1189.0       True   
300750.SZ 2019-06-11    67.269997    74.180000    245.0   243.0       True   
600570.SH 2019-06-11  1190.318726  1338.804565   2232.0  1288.0       True   
002714.SZ 2019-06-11   458.771515   497.444794   1287.0  1282.0       True   
300347.SZ 2019-06-20   562.727539   685.353455   1531.0  1197.0       True   
002371.SZ 2019-06-21   316.765625   343.510498   2177.0  1262.0       True   
          2019-06-26   321.555023   343.510498   2177.0  1265.0       True   
300033.SZ 2019-08-15   673.217957  1119.958496   2335.0  1371.0       True   
          2019-08-19   768.158936   806.912109   2324.0  1373.0       True   
          2019-12-05   819.081848  1068.517822   2400.0  1445.0       True   
002841.SZ 2019-12-16   143.713913   145.900803    711.0   708.0       True   
603658.SH 2020-01-09   105.980721   138.291672    830.0   817.0       True   
600763.SH 2020-01-16   255.965836   263.838745   2282.0  1319.0       True   
603259.SH 2020-02-03   129.684372   167.459656    438.0   423.0       True   
603288.SH 2020-02-03   400.570007   423.197327   1464.0  1458.0       True   
300033.SZ 2020-02-04   849.980896  1292.923706   2442.0  1481.0       True   
600763.SH 2020-02-04   232.443741   277.604248   2309.0  1326.0       True   
000858.SZ 2020-02-04  2007.619873  2272.646484   2397.0  1422.0       True   
300033.SZ 2020-02-06   882.088135  1292.923706   2442.0  1483.0       True   
603833.SH 2020-02-12   115.041687   125.282959    698.0   694.0       True   
          2020-03-02   118.750084   122.936005    709.0   707.0       True   
000596.SZ 2020-03-05   442.965576   463.608002   2461.0  1504.0       True   
          2020-03-10   433.152008   450.072021   2464.0  1507.0       True   
300709.SZ 2020-03-10   118.100395   119.112396    582.0   582.0       True   
300033.SZ 2020-03-13  1019.925232  1020.184143   2457.0  1509.0       True   
000858.SZ 2020-03-20  1801.488037  2659.446777   2457.0  1455.0      False   
603288.SH 2020-03-20   380.639191   531.019043   

In [202]:
tmp.sort_values('date').tail(50)

buy_at          max  max_idx     idx  is_selled  \
code      date                                                               
300628.SZ 2019-11-15   563.165771   602.316833    653.0   651.0       True   
000858.SZ 2019-11-15  2311.101318  2348.170410   2337.0  1372.0       True   
600570.SH 2019-11-19  1438.106934  1473.077759   2334.0  1397.0       True   
          2019-11-20  1432.496582  1469.711548   2336.0  1398.0       True   
300760.SZ 2019-11-21   192.840500   193.142593    270.0   270.0       True   
600570.SH 2019-11-21  1441.660034  1469.711548   2336.0  1399.0       True   
300760.SZ 2019-11-22   184.432037   189.195160    274.0   271.0       True   
600570.SH 2019-11-22  1433.057617  1433.805664   2337.0  1400.0       True   
300760.SZ 2019-11-25   185.287994   189.195160    274.0   272.0       True   
          2019-11-26   185.539749   189.195160    274.0   273.0       True   
          2019-11-27   185.590103   189.195160    274.0   274.0       True   
          2019-11-28   184.784500   189.265640    285.0   275.0       True   
600536.SH 2019-12-02   386.681000   445.106903   2396.0  1439.0       True   
603444.SH 2019-12-03   314.257507   357.086700    721.0   710.0       True   
300760.SZ 2019-12-06   187.080460   189.265640    285.0   281.0       True   
600519.SH 2019-12-11  8460.763672  8749.864258   2423.0  1450.0       True   
300601.SZ 2019-12-23   129.709000   135.582901    710.0   705.0       True   
603259.SH 2019-12-24   129.487106   132.375549    402.0   401.0       True   
300601.SZ 2019-12-25   129.814697   135.582901    710.0   707.0       True   
603259.SH 2019-12-25   128.148544   132.375549    402.0   402.0       True   
603444.SH 2019-12-26   325.813385   489.152496    760.0   727.0       True   
300558.SZ 2019-12-26    67.905701    79.489014    783.0   768.0       True   
603444.SH 2019-12-27   321.975861   489.152496    760.0   728.0       True   
300601.SZ 2019-12-30   131.400192   135.582901    710.0   710.0       True   
603444.SH 2019-12-30   326.051208   489.152496    760.0   729.0       True   
002821.SZ 2020-01-08   270.588562   404.557129    793.0   767.0       True   
300033.SZ 2020-01-15   949.323608   982.898193   2426.0  1473.0       True   
          2020-01-16   948.460510   982.898193   2426.0  1474.0       True   
          2020-01-17   955.710632   982.898193   2426.0  1475.0       True   
          2020-01-21   933.528931   982.898193   2426.0  1477.0       True   
          2020-01-22   957.954651   982.898193   2426.0  1478.0       True   
300676.SZ 2020-03-09    82.573997    82.573997    635.0   635.0       True   
600436.SH 2020-03-31   808.944397   977.892029   2485.0  1516.0       True   
          2020-04-02   797.229980   977.892029   2485.0  1518.0       True   
002714.SZ 2020-04-15   919.027893  1036.353882   1496.0  1489.0       True   
          2020-04-17   937.393311  1036.353882   1496.0  1491.0       True   
          2020-04-20   939.594116  1036.353882   1496.0  1492.0       True   
          2020-04-21   933.447021  1036.353882   1496.0  1493.0       True   
600570.SH 2020-04-24  1815.306030  2066.460449   2453.0  1502.0      False   
002714.SZ 2020-05-06   938.379883   959.097839   1501.0  1501.0       True   
          2020-05-07   940.353027   943.312744   1502.0  1502.0       True   
          2020-05-08   925.858032   928.134766   1503.0  1503.0       True   
300033.SZ 2020-05-13  1039.530518  1048.229492   2499.0  1548.0       True   
300750.SZ 2020-05-14   140.219879   155.710785    471.0   468.0       True   
300033.SZ 2020-05-14  1029.091797  1048.229492   2499.0  1549.0       True   
          2020-05-15  1017.870056  1048.229492   2499.0  1550.0       True   
          2020-05-19  1022.045532  1023.437378   2500.0  1552.0       True   
300628.SZ 2020-05-20   748.310547   752.176270    773.0   773.0       True   
300482.SZ 2020-05-22   320.514496   334.310730   1185.0  1184.0       True   
603444.SH 2020-05-28   431.976807   447.314728   

In [116]:
target_ss = df[target_col].fillna(0)
%time _ = target_ss.groupby('code').rolling(20).apply(lambda x:np.argmax(x))
import time
t0 = time.time()
result = target_ss.rolling(20).apply(lambda x:np.argmax(x))
df_check = pd.DataFrame(index=df.index)
df_check['mark']=1
mask = df_check.groupby('code')['mark'].shift(k - 1).isnull()
result.loc[mask]=np.nan
print(time.time()-t0)


C:\Users\dell-pc\Anaconda3.6\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  """Entry point for launching an IPython kernel.


Wall time: 11.5 s


C:\Users\dell-pc\Anaconda3.6\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  """


11.123180627822876


In [151]:
((df['p20max_close']/df['close']-1)).head(100)

df[['close','p20max_close','p20max_p3mean_amt','p3mean_amt']].head(40)

code       date      
000001.SZ  2014-01-02         NaN
           2014-01-03         NaN
           2014-01-06         NaN
           2014-01-07         NaN
           2014-01-08         NaN
           2014-01-09         NaN
           2014-01-10         NaN
           2014-01-13         NaN
           2014-01-14         NaN
           2014-01-15         NaN
           2014-01-16         NaN
           2014-01-17         NaN
           2014-01-20         NaN
           2014-01-21         NaN
           2014-01-22         NaN
           2014-01-23         NaN
           2014-01-24         NaN
           2014-01-27         NaN
           2014-01-28         NaN
           2014-01-29    0.057958
           2014-01-30    0.046491
           2014-02-07    0.038664
           2014-02-10    0.021608
           2014-02-11    0.000000
           2014-02-12    0.005025
           2014-02-13    0.000000
           2014-02-14    0.011590
           2014-02-17    0.020886
           2014-02-18    0

close  p20max_close  p20max_p3mean_amt  p3mean_amt
code      date                                                               
000001.SZ 2014-01-02  714.072998           NaN                NaN         NaN
          2014-01-03  696.556946           NaN                NaN         NaN
          2014-01-06  681.376282           NaN                NaN    6.440451
          2014-01-07  679.040833           NaN                NaN    5.766298
          2014-01-08  686.631165           NaN                NaN    5.372314
          2014-01-09  690.134338           NaN                NaN    5.030948
          2014-01-10  690.134338           NaN                NaN    5.219315
          2014-01-13  677.289246           NaN                NaN    5.289112
          2014-01-14  684.295654           NaN                NaN    4.768594
          2014-01-15  681.376282           NaN                NaN    4.422745
          2014-01-16  682.544006           NaN                NaN    3.724920
          2014-01-17  670.282715           NaN                NaN    3.947280
          2014-01-20  659.773132           NaN                NaN    3.955350
          2014-01-21  663.276306           NaN                NaN    3.800520
          2014-01-22  688.382751           NaN                NaN    4.587520
          2014-01-23  683.127930           NaN                NaN    4.888663
          2014-01-24  680.792419           NaN                NaN    5.768454
          2014-01-27  665.027954           NaN                NaN    5.051187
          2014-01-28  671.450500           NaN                NaN    5.073574
          2014-01-29  674.953735    714.072998           6.440451    4.387105
          2014-01-30  665.611816    696.556946           6.440451    3.801356
          2014-02-07  664.444092    690.134338           6.440451    3.881106
          2014-02-10  675.537598    690.134338           5.768454    4.909218
          2014-02-11  700.644043    700.644043           9.032334    9.032334
          2014-02-12  697.140747    700.644043          10.237650   10.237650
          2014-02-13  713.489197    713.489197          12.939544   12.939544
          2014-02-14  705.314941    713.489197          12.939544   10.047386
          2014-02-17  698.892395    713.489197          12.939544    9.700279
          2014-02-18  680.792419    713.489197          12.939544    7.099177
          2014-02-19  700.644043    713.489197          12.939544    8.685369
          2014-02-20  686.631165    713.489197          12.939544    9.323386
          2014-02-21  677.289246    713.489197          12.939544    8.854837
          2014-02-24  651.598938    713.489197          12.939544    7.431637
          2014-02-25  643.424805    713.489197          12.939544    6.406989
          2014-02-26  648.679565    713.489197          12.939544    6.051933
          2014-02-27  656.269897    713.489197          12.939544    5.621958
          2014-02-28  649.847351    713.489197          12.939544    5.228416
          2014-03-03  644.592468    713.489197          12.939544    5.285521
          2014-03-04  641.089233    713.489197          12.939544    5.129665
          2014-03-05  624.740906    713.489197          12.939544    5.402005

In [10]:
mask_date = df.index.get_level_values('date')>='2016-01-01'
